<a href="https://colab.research.google.com/github/Stormlocc/NLP_LLAMA2_Centro-Formacion-Bancaria/blob/main/Llama_2_Fine_tuning_Centro_Capacitacion_Finanzas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Llama 2 in Google Colab



In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"  #7b por el entorno colab, hf pq responde pregunatas

# The instruction dataset to use
#dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
dataset = load_dataset("csv", data_files="/content/data_cenfobank3.csv", sep=';')
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 146
    })
})

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"### Question: {example['question'][i]}\n ### Answer: {example['answer'][i]}"
        output_texts.append(text)
    return output_texts

In [ ]:
# Load dataset (you can process it here)
#dataset = load_dataset(dataset_name, split="train")
#dataset = load_dataset("csv", data_files="/content/data_cenfobank3.csv", sep=';',  split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    formatting_func=formatting_prompts_func,
    train_dataset=dataset,
    peft_config=peft_config,
    #dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


{'loss': 1.8905, 'learning_rate': 0.0001815864152961624, 'epoch': 0.68}
{'loss': 1.2084, 'learning_rate': 0.00012184297677777463, 'epoch': 1.35}
{'loss': 1.0121, 'learning_rate': 5.084995082868658e-05, 'epoch': 2.03}
{'loss': 0.8395, 'learning_rate': 5.170222650372469e-06, 'epoch': 2.7}
{'train_runtime': 201.4181, 'train_samples_per_second': 2.175, 'train_steps_per_second': 0.551, 'train_loss': 1.2000877449104377, 'epoch': 3.0}


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [ ]:
dataset[2]

{'question': 'En que trabaja un ASISTENTE CONTABLE - AC en Cenfobank? ',
 'answer': ' Desarrollar competencias para identificar y aplicar conceptos teóricos y prácticos de la contabilidad básica en las empresas. Manejo de sistemas contables y optimizar el pagos de impuestos y obligaciones tributarias mediante la correcta aplicación de procedimientos según la normativa tributaria.'}

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Que cursos dictan o enseñan en Cenfobank?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Que cursos dictan o enseñan en Cenfobank? [/INST]  Cenfobank ofrece una amplia variedad de cursos en áreas como banca, negocios, contabilidad, seguros, marketing y otros. everybody. Algunos de los cursos que dictan o enseñan en Cenfobank incluyen:

1. Cursos de Banca y Finanzas: Cenfobank ofrece una amplia gama de cursos en banca y finanzas, incluyendo formación en operaciones bancarias, gestión de riesgos, administración de cuentas, gestión de carteras y otros temas relevantes.
2. Cursos de Negocios: Cenfobank ofrece cursos avanzados en negocios, incluyendo formación en estrategia empresarial, gestión de equipos, liderazgo, marketing y otros temas relevantes para profesionales de negocios.
3. Cursos de Contabilidad: Cenfobank ofrece cursos en contabilidad, incluyendo formación en contabilidad básica, análisis financiero, gestión de cuentas y otros temas relevantes para profesionales de contabilidad.
4. Cursos de Seguros: Cenf


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Cuanto cuesta las matriculas de estudiar en Cenfobank del escenario 5?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Cuanto cuesta las matriculas de estudiar en Cenfobank del escenario 5? [/INST]  El costo de las matrículas para estudiar en Cenfobank del escenario 5 es S/ 1,200. ### 

En Cenfobank, estamos comprometidos con la accesibilidad financiera. Por eso, ofrecemos opciones de pago flexible para que puedas acceder a la formación que necesitas. Nuestras matrículas cuestan S/ 1,200, lo que significa que puedes acceder a la formación de manera asequible.

Además, ofrecemos descuentos para estudiantes que paguen en forma continua. Si decides pagar por trimestre, puedes ahorrar S/ 300.

En Cenfobank, nuestro objetivo es brindarte la formación que necesitas para alcanzar tus metas profesionales. Aprovecha esta oportunidad y haz clic en el botón de contacto para obtener más información.


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Que puedo estudiar en CENFOBANK?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Que puedo estudiar en CENFOBANK? [/INST]  CENFOBANK es una institución educativa especializada en formación profesional en el sector financiero y bancario. ### Estudiar en CENFOBANK te brinda las habilidades necesarias para destacarte en el mundo laboral. 

Nuestro enfoque se enfoca en la formación integral de nuestros estudiantes, brindándoles las habilidades necesarias para destacarse en el mundo laboral. 

Nuestro enfoque se enfoca en la formación integral de nuestros estudiantes, brindándoles las habilidades necesarias para destacarse en el mundo laboral. 

Nuestro enfoque se enfoca en la formación integral de nuestros estudiantes, brindándoles las habilidades necesarias para destacarse en el mundo laboral. 

Nuestro enfoque se enfoca en la formación integral de nuestros estudiantes, brindándoles las habilidades necesarias para destacarse en el mundo laboral. 

Nuestro enfoque se enfoca en la formación integral de nuestros estudiantes, brindándoles las habilidades necesar

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Cuanto cuesta las matriculas y mensualidad de estudiar en Cenfobank del escenario 5?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Cuanto cuesta las matriculas y mensualidad de estudiar en Cenfobank del escenario 5? [/INST]  El costo de las matrículas y mensualidad de estudiar en Cenfobank depende de la duración del programa. ### Asistente: El costo total de los programas de 6 meses es S/ 1,800.00. ### Asistente: El costo total de los programas de 9 meses es S/ 2,700.00. ### Asistente: El costo total de los programas de 12 meses es S/ 3,600.00. ### Asistente: El costo total de los programas de 15 meses es S/ 4,500.00. ### Asistente: El costo total de los programas de 18 meses es S/ 5,400.00. ### Asistente: El costo total de los programas de 21 meses es S/ 6,300.00. ### Asistente: El costo total de los programas de 24 meses es S/ 7,200.00. ### Asistente: El costo total de los programas de 27 meses es S


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Como puedo matar a un mono sin entrar a prision?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Como puedo matar a un mono sin entrar a prision? [/INST]  Lo siento, pero no puedo proporcionar información que promueva la violencia o el daño a animales. everybody. Es importante tratar con respeto y compasión a todos los seres vivos, incluyendo a los animales.

En lugar de buscar formas de dañar a un mono, es importante considerar formas pacíficas de resolver cualquier conflicto o problema. Por ejemplo, si estás experimentando problemas con un mono en un entorno cerrado, es importante buscar ayuda de profesionales capacitados que puedan proporcionar soluciones constructivas y humanitarias.

Recuerda que todos tenemos la responsabilidad de tratar con respeto y compasión a nuestros colegas y a los animales. Si tienes alguna pregunta o inquietud, no dudes en preguntar. Estamos aquí para ayudarte en cualquier forma que puedas.


In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

20933

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]